In [ ]:
import gc
from functools import partial

import torch

from diffusers import IFPipeline, IFSuperResolutionPipeline
from wandb_addons.diffusers import IFCallback

In [ ]:
pipeline = IFPipeline.from_pretrained(
    "DeepFloyd/IF-I-XL-v1.0", variant="fp16", torch_dtype=torch.float16
)
pipeline.enable_model_cpu_offload()

In [ ]:
prompt = 'a photo of a kangaroo wearing an orange hoodie and blue sunglasses standing in front of the eiffel tower holding a sign that says "very deep learning"'
prompt_embeds, negative_embeds = pipeline.encode_prompt(prompt)
num_images_per_prompt = 2
num_inference_steps = 100
configs = {"guidance_scale": 7.0}

callback = IFCallback(
    pipeline=pipeline,
    prompt=prompt,
    wandb_project="diffusers-2",
    wandb_entity="geekyrakshit",
    weave_mode=False,
    num_inference_steps=num_inference_steps,
    num_images_per_prompt=num_images_per_prompt,
    configs=configs
)

image = pipeline(
    prompt_embeds=prompt_embeds,
    negative_prompt_embeds=negative_embeds,
    output_type="pt",
    callback=partial(callback, end_experiment=False),
    **configs,
).images

In [ ]:
del pipeline
gc.collect()
torch.cuda.empty_cache()

In [ ]:
pipeline_2 = IFSuperResolutionPipeline.from_pretrained(
    "DeepFloyd/IF-II-L-v1.0", text_encoder=None, variant="fp16", torch_dtype=torch.float16
)
pipeline_2.enable_model_cpu_offload()

In [ ]:
num_inference_steps = 50

callback.add_stage(pipeline_2, num_inference_steps=num_inference_steps)

image = pipeline_2(
    image=image,
    prompt_embeds=prompt_embeds,
    negative_prompt_embeds=negative_embeds,
    num_inference_steps=num_inference_steps,
    output_type="pt",
    callback=callback,
).images

In [ ]:
del pipeline_2
gc.collect()
torch.cuda.empty_cache()